__author__ = 'Ricardo Pasquini'

# Counting tweets at the hexagon level. Complete procedure

Overview: This code counts georeferenced tweets at the hexagon level. Hexagons are uniquely identified in space. We use Uber's H3 library for this purpose.  
The code  also performs a home location prediction for each user. This allows later counting tweets by whether they were tweeted by residents, residents of neighbors hexagons, or non-residents.
The project was designed to be scalable to millions of tweets. The process is done with MongoDB.

By default the project generates the following collections on the Mongo database:

tweets: Collection containing tweets. This collection is modified by incorporating the corresponding hex id. \
users: Collection containing users. This collection is modified by incorporating the home location data.\
hexcounts: Collection containing counts of tweets.


In [65]:
import sys
sys.path.append("../") 
import databasepopulation
import communicationwmongo as commu
import home_location as home
import analysis as a
import pymongo

In [105]:
import importlib
importlib.reload(databasepopulation)
importlib.reload(home)
importlib.reload(a)

<module 'analysis' from '../analysis.py'>

# Counting tweets at the hexagon level. Complete procedure

This version of the code assumes that a different database will be used for each city.


If necessary check your current location with the following command:

In [10]:
 !pwd

/mnt/c/Users/Richard/Drive/projects/twitter/notebooks


# 0. Connect to Mongo and define a specific database

By default connect to Mongo running on a localhost. 
Requires manually changing the parameters if working on a cloud environment

I will use the name buenosaires as the db example. Recall each database for each city.


In [ ]:
db=commu.connecttoLocaldb(database='buenosaires')

# 1. Raw data to Mongo
This function populates the files containing tweets into Mongo.
Assumes files are named like: ba_2012.csv, etc.
Choose start and end years.


In [12]:
databasepopulation.populatetweets(db, path='/mnt/d/twitter/', cityprefix='ba', yearstart=2012, yearend=2012)


/home/rickhunter/miniconda3/envs/twitteranalysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# 2. Adding Hexids to tweets 
This function includes hexagons ids to each tweet.  

In [40]:
databasepopulation.addhexjob(db)

 iter: 1  time: 0.22681570053100586
 iter: 2  time: 0.21671772003173828
 iter: 3  time: 0.2405691146850586
 iter: 4  time: 0.31368541717529297
 iter: 5  time: 0.27761077880859375
 iter: 6  time: 0.26112890243530273
 iter: 7  time: 0.22281408309936523
 iter: 8  time: 0.1968994140625
 iter: 9  time: 0.3073241710662842
 iter: 10  time: 0.22986364364624023
 iter: 11  time: 0.6346921920776367
 iter: 12  time: 0.3234543800354004
 iter: 13  time: 0.26929235458374023
 iter: 14  time: 0.21526741981506348
 iter: 15  time: 0.18408441543579102
 iter: 16  time: 0.20883941650390625
 iter: 17  time: 0.1782221794128418
 iter: 18  time: 0.20904302597045898
 iter: 19  time: 0.18248629570007324
 iter: 20  time: 0.28961730003356934
 iter: 21  time: 0.20610713958740234
 iter: 22  time: 0.1990804672241211
 iter: 23  time: 0.2217543125152588
 iter: 24  time: 0.34151649475097656
 iter: 25  time: 0.18855738639831543
 iter: 26  time: 0.2612776756286621
 iter: 27  time: 0.19443297386169434
 iter: 28  time: 0.248

 iter: 222  time: 0.24240994453430176
 iter: 223  time: 0.18688249588012695
 iter: 224  time: 0.18803095817565918
 iter: 225  time: 0.23665189743041992
 iter: 226  time: 0.1896977424621582
 iter: 227  time: 0.18918442726135254
 iter: 228  time: 0.23843836784362793
 iter: 229  time: 0.19916486740112305
 iter: 230  time: 0.18266034126281738
 iter: 231  time: 0.1859571933746338
 iter: 232  time: 0.19191622734069824
 iter: 233  time: 0.17608118057250977
 iter: 234  time: 0.1799468994140625
 iter: 235  time: 0.21285176277160645
 iter: 236  time: 0.17495274543762207
 iter: 237  time: 0.2613091468811035
 iter: 238  time: 0.24863028526306152
 iter: 239  time: 0.17458248138427734
 iter: 240  time: 0.19231867790222168
 iter: 241  time: 0.16782402992248535
 iter: 242  time: 0.26495909690856934
 iter: 243  time: 0.16640567779541016
 iter: 244  time: 0.20889687538146973
 iter: 245  time: 0.170180082321167
 iter: 246  time: 0.2502164840698242
 iter: 247  time: 0.1844339370727539
 iter: 248  time: 0.

 iter: 442  time: 0.18553900718688965
 iter: 443  time: 0.16589117050170898
 iter: 444  time: 0.18423199653625488
 iter: 445  time: 0.19298243522644043
 iter: 446  time: 0.17166829109191895
 iter: 447  time: 0.1706836223602295
 iter: 448  time: 0.17926263809204102
 iter: 449  time: 0.17778944969177246
 iter: 450  time: 0.1854391098022461
 iter: 451  time: 0.17664837837219238
 iter: 452  time: 0.20370244979858398
 iter: 453  time: 0.16173148155212402
 iter: 454  time: 0.1796102523803711
 iter: 455  time: 0.17112278938293457
 iter: 456  time: 0.17576169967651367
 iter: 457  time: 0.1693274974822998
 iter: 458  time: 0.17146086692810059
 iter: 459  time: 0.19830322265625
 iter: 460  time: 0.18302369117736816
 iter: 461  time: 0.16973423957824707
 iter: 462  time: 0.18474268913269043
 iter: 463  time: 0.1646871566772461
 iter: 464  time: 0.18137121200561523
 iter: 465  time: 0.16420483589172363
 iter: 466  time: 0.2139132022857666
 iter: 467  time: 0.16913628578186035
 iter: 468  time: 0.1

 iter: 660  time: 0.24750280380249023
 iter: 661  time: 0.18868303298950195
 iter: 662  time: 0.27356576919555664
 iter: 663  time: 0.20892024040222168
 iter: 664  time: 0.18094396591186523
 iter: 665  time: 0.19944405555725098
 iter: 666  time: 0.17201948165893555
 iter: 667  time: 0.17838358879089355
 iter: 668  time: 0.18673467636108398
 iter: 669  time: 0.29266905784606934
 iter: 670  time: 0.1799302101135254
 iter: 671  time: 0.1859126091003418
 iter: 672  time: 0.21998000144958496
 iter: 673  time: 0.18451523780822754
 iter: 674  time: 0.20268583297729492
 iter: 675  time: 0.4348313808441162
 iter: 676  time: 0.2006683349609375
 iter: 677  time: 0.18675923347473145
 iter: 678  time: 0.22118544578552246
 iter: 679  time: 0.18838143348693848
 iter: 680  time: 0.1833937168121338
 iter: 681  time: 0.18024086952209473
 iter: 682  time: 0.23058724403381348
 iter: 683  time: 0.2419421672821045
 iter: 684  time: 0.2447953224182129
 iter: 685  time: 0.1804502010345459
 iter: 686  time: 0.

 iter: 878  time: 0.2401742935180664
 iter: 879  time: 0.24909090995788574
 iter: 880  time: 0.17066407203674316
 iter: 881  time: 0.18466496467590332
 iter: 882  time: 0.16529321670532227
 iter: 883  time: 0.17844891548156738
 iter: 884  time: 0.19143271446228027
 iter: 885  time: 0.17539191246032715
 iter: 886  time: 0.19785785675048828
 iter: 887  time: 0.20235371589660645
 iter: 888  time: 0.16669797897338867
 iter: 889  time: 0.19983649253845215
 iter: 890  time: 0.2625386714935303
 iter: 891  time: 0.35063862800598145
 iter: 892  time: 0.2893853187561035
 iter: 893  time: 0.3216822147369385
 iter: 894  time: 0.25447607040405273
 iter: 895  time: 0.1843092441558838
 iter: 896  time: 0.24501729011535645
 iter: 897  time: 0.1995983123779297
 iter: 898  time: 0.17865252494812012
 iter: 899  time: 0.5196187496185303
 iter: 900  time: 0.20398855209350586
 iter: 901  time: 0.2754678726196289
 iter: 902  time: 0.16682147979736328
 iter: 903  time: 0.18809866905212402
 iter: 904  time: 0.

...

 iter: 2589  time: 0.3457818031311035
 iter: 2590  time: 0.4403798580169678
 iter: 2591  time: 0.23970413208007812
 iter: 2592  time: 0.6522314548492432
 iter: 2593  time: 0.5329828262329102
 iter: 2594  time: 0.37474679946899414
 iter: 2595  time: 0.4575793743133545
 iter: 2596  time: 0.39900803565979004
 iter: 2597  time: 0.3899216651916504
 iter: 2598  time: 0.5385138988494873
 iter: 2599  time: 0.6592814922332764
 iter: 2600  time: 0.5538485050201416
 iter: 2601  time: 1.3070571422576904
 iter: 2602  time: 1.7791798114776611
 iter: 2603  time: 0.5399734973907471
 iter: 2604  time: 0.5213987827301025
 iter: 2605  time: 0.40697646141052246
 iter: 2606  time: 0.3989729881286621
 iter: 2607  time: 0.20500922203063965
 iter: 2608  time: 0.2833280563354492
 iter: 2609  time: 0.262709379196167
 iter: 2610  time: 0.3498823642730713
 iter: 2611  time: 0.5205497741699219
 iter: 2612  time: 0.18868136405944824
 iter: 2613  time: 0.20215654373168945
 iter: 2614  time: 0.255281925201416
 iter: 

 iter: 2803  time: 0.28143739700317383
 iter: 2804  time: 0.20099711418151855
 iter: 2805  time: 0.6737499237060547
 iter: 2806  time: 0.17410063743591309
 iter: 2807  time: 0.184617280960083
 iter: 2808  time: 0.172318696975708
 iter: 2809  time: 0.21189117431640625
 iter: 2810  time: 0.18150854110717773
 iter: 2811  time: 0.2300434112548828
 iter: 2812  time: 0.17081022262573242
 iter: 2813  time: 0.19423604011535645
 iter: 2814  time: 0.17355680465698242
 iter: 2815  time: 0.17871928215026855
 iter: 2816  time: 0.27170276641845703
 iter: 2817  time: 0.19618916511535645
 iter: 2818  time: 0.21248650550842285
 iter: 2819  time: 0.17307543754577637
 iter: 2820  time: 0.20014643669128418
 iter: 2821  time: 0.9490196704864502
 iter: 2822  time: 0.17913174629211426
 iter: 2823  time: 0.19629120826721191
 iter: 2824  time: 0.179046630859375
 iter: 2825  time: 0.22170543670654297
 iter: 2826  time: 0.1780991554260254
 iter: 2827  time: 0.22674345970153809
 iter: 2828  time: 0.17228245735168

 iter: 3016  time: 0.25509166717529297
 iter: 3017  time: 0.17358779907226562
 iter: 3018  time: 0.19652390480041504
 iter: 3019  time: 0.19690728187561035
 iter: 3020  time: 0.22900676727294922
 iter: 3021  time: 0.21040558815002441
 iter: 3022  time: 0.1775808334350586
 iter: 3023  time: 0.18794465065002441
 iter: 3024  time: 0.1671898365020752
 iter: 3025  time: 0.17818212509155273
 iter: 3026  time: 0.24502205848693848
 iter: 3027  time: 0.1746072769165039
 iter: 3028  time: 0.47936582565307617
 iter: 3029  time: 0.18770051002502441
 iter: 3030  time: 0.18492388725280762
 iter: 3031  time: 0.30167269706726074
 iter: 3032  time: 0.18676543235778809
 iter: 3033  time: 0.35594725608825684
 iter: 3034  time: 0.19105291366577148
 iter: 3035  time: 0.20425128936767578
 iter: 3036  time: 0.17696547508239746
 iter: 3037  time: 0.18201732635498047
 iter: 3038  time: 0.17446613311767578
 iter: 3039  time: 0.23470759391784668
 iter: 3040  time: 0.5087947845458984
 iter: 3041  time: 0.18017172

 iter: 3229  time: 0.18213629722595215
 iter: 3230  time: 0.17753839492797852
 iter: 3231  time: 0.21648216247558594
 iter: 3232  time: 0.1900463104248047
 iter: 3233  time: 0.2940363883972168
 iter: 3234  time: 0.20749759674072266
 iter: 3235  time: 0.489168643951416
 iter: 3236  time: 0.22326254844665527
 iter: 3237  time: 0.2501354217529297
 iter: 3238  time: 0.28455185890197754
 iter: 3239  time: 0.1779944896697998
 iter: 3240  time: 0.2731297016143799
 iter: 3241  time: 0.18101143836975098
 iter: 3242  time: 0.3875765800476074
 iter: 3243  time: 0.4037463665008545
 iter: 3244  time: 0.19993948936462402
 iter: 3245  time: 0.3546180725097656
 iter: 3246  time: 0.30118560791015625
 iter: 3247  time: 0.1827096939086914
 iter: 3248  time: 0.20051980018615723
 iter: 3249  time: 0.17448163032531738
 iter: 3250  time: 0.1881260871887207
 iter: 3251  time: 0.18726801872253418
 iter: 3252  time: 0.22204089164733887
 iter: 3253  time: 0.18149590492248535
 iter: 3254  time: 0.1959056854248047

 iter: 3443  time: 0.2821383476257324
 iter: 3444  time: 0.41986680030822754
 iter: 3445  time: 0.5043478012084961
 iter: 3446  time: 0.34568095207214355
 iter: 3447  time: 0.3950169086456299
 iter: 3448  time: 0.3397800922393799
 iter: 3449  time: 0.2757868766784668
 iter: 3450  time: 0.26758337020874023
 iter: 3451  time: 0.2037038803100586
 iter: 3452  time: 0.22530293464660645
 iter: 3453  time: 0.20925545692443848
 iter: 3454  time: 0.21829533576965332
 iter: 3455  time: 0.6485931873321533
 iter: 3456  time: 0.26557111740112305
 iter: 3457  time: 0.18237972259521484
 iter: 3458  time: 0.3568897247314453
 iter: 3459  time: 0.3628087043762207
 iter: 3460  time: 0.45288634300231934
 iter: 3461  time: 1.176145315170288
 iter: 3462  time: 0.23463654518127441
 iter: 3463  time: 0.1844937801361084
 iter: 3464  time: 0.19559502601623535
 iter: 3465  time: 0.18024516105651855
 iter: 3466  time: 0.19373774528503418
 iter: 3467  time: 0.2921113967895508
 iter: 3468  time: 0.16890692710876465

In [ ]:
# the following is a check and a fix because I think the previous process did not run completely, probably a bug.
# are there any other documents that were not processed?
len(list(db.tweets.find({ "hex" : { "$exists" : False } })))

In [80]:
#Process any missing document
cursor=db.tweets.find({ "hex" : { "$exists" : False } })
df = pd.DataFrame(list(cursor))
requests = databasepopulation.add_hexs_and_prepare_bulk_request(df, dataformat='raw')
try:
    db.tweets.bulk_write(requests, ordered=False)
except BulkWriteError as bwe:
    print(bwe.details)

# 3. Database performance task: Add indexes to tweets

In [51]:
databasepopulation.create_indexes(db)

'u_id_1'

# 4. Populate Users Collection

In [60]:
databasepopulation.populate_users_collection(db)

# 5. Find Home Job for each user id in the database

In [ ]:
home.job_findhomeandpopulate_hex9(db)

In [99]:
print('Users with home location identified', db.users.count_documents({'hex9': { '$exists': True} }))


Users with home location identified 16467


# 6 Generate a Hex-level collection including all hexs in the database

By default the collection include as ids only the ids of hexagons at resolution 9
Note that many types of hexogons could coexist in a same collection because hex identifiers are unique. This is not implemented in the current version of the code, though.

In [102]:
databasepopulation.populate_hexcounts_collection(db)


# 7 Count tweets in each hex by residents and non-residents

In [103]:
import analysis as a

In [115]:
a.countandpopulatejob(db)

Hexagons pending to analyze.. 8733
iter: 50
iter: 100
iter: 150
iter: 200
iter: 250
iter: 300
iter: 350
iter: 400
iter: 450
iter: 500
iter: 550
iter: 600
iter: 650
iter: 700
iter: 750
iter: 800
iter: 850
iter: 900
iter: 950
iter: 1000
iter: 1050
iter: 1100
iter: 1150
iter: 1200
iter: 1250
iter: 1300
iter: 1350
iter: 1400
iter: 1450
iter: 1500
iter: 1550
iter: 1600
iter: 1650
iter: 1700
iter: 1750
iter: 1800
iter: 1850
iter: 1900
iter: 1950
iter: 2000
iter: 2050
iter: 2100
iter: 2150
iter: 2200
iter: 2250
iter: 2300
iter: 2350
iter: 2400
iter: 2450
iter: 2500
iter: 2550
iter: 2600
iter: 2650
iter: 2700
iter: 2750
iter: 2800
iter: 2850
iter: 2900
iter: 2950
iter: 3000
iter: 3050
iter: 3100
iter: 3150
iter: 3200
iter: 3250
iter: 3300
iter: 3350
iter: 3400
iter: 3450
iter: 3500
iter: 3550
iter: 3600
iter: 3650
iter: 3700
iter: 3750
iter: 3800
iter: 3850
iter: 3900
iter: 3950
iter: 4000
iter: 4050
iter: 4100
iter: 4150
iter: 4200
iter: 4250
iter: 4300
iter: 4350
iter: 4400
iter: 4450
iter: 

# 8 Query to the DB to create the counts dataframe 

In [116]:
df=a.hexcountsresults_to_df(db, save=False)
# if save=True then saves resulting dataframe a pickle in ./hexcountsdf.pkl

In [118]:
df

,time,_id,nonresidents,nonresidentsandnonneighbors,residents,totalcounts
0,2013-03-31,89c2e3c529bffff,1.0,1.0,NaN,1
1,2013-03-31,89c2e38621bffff,1.0,1.0,NaN,1
2,2013-03-31,89c2e398577ffff,1.0,1.0,NaN,1
3,2013-03-31,89c2e333633ffff,1.0,1.0,NaN,1
4,2012-12-31,89c2e3059cbffff,1.0,1.0,NaN,1
5,2013-03-31,89c2e38a88fffff,1.0,1.0,NaN,1
6,2012-12-31,89c2e3a346bffff,1.0,1.0,NaN,1
7,2012-12-31,89c2e3b3673ffff,1.0,1.0,NaN,1
8,2012-12-31,89c2e33310fffff,1.0,1.0,NaN,1
9,2012-12-31,89c2e3d1133ffff,1.0,1.0,NaN,1
